

`collections` 是 Python 的内建库，提供了一系列的容器数据类型。这些数据类型可以用来在程序中存储和操作数据。`collections` 库包含了许多有用的数据类型，例如：

`Counter`:统计可迭代对象中元素的出现次数
`deque`:双端队列，支持从队列的两端插入和删除元素
`OrderedDict`:有序字典
`defaultdict`:带有默认值的字典
`namedtuple`:带有名称的元组

deque（双端队列）是 Python collections 库中的一种数据类型，它可以在队列的两端插入和删除元素。

deque 类似于列表，但是它支持从队列的两端插入和删除元素，而列表只能从一端插入，从另一端删除。因此，deque 通常用于实现队列或栈的数据结构。



In [1]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random

from collections import deque

### Huber loss

Huber损失是用于稳健回归的损失函数

\begin{equation}
L_\delta = 
\left\{
             \begin{array}{lr}
             \frac{1}{2} (y-f(x))^{2} & |y-f(x)|\le \delta\\
             \delta \cdot (|y-f(x)|-\frac{1}{2}\delta) & otherwise
             \end{array}
\right.
\end{equation}

References: 
> https://en.wikipedia.org/wiki/Huber_loss 

> https://www.tensorflow.org/api_docs/python/tf/losses/huber_loss

### 经验回放

经验回放（*experience replay*）是一种在强化学习（*reinforcement learning*）中常用的方法，用于解决数据相关性问题。

在强化学习中，Agent 通过在环境中进行探索来学习最优的行为策略。在这个过程中，*Agent* 会经历许多不同的状态和执行许多不同的动作，并获得相应的奖励。这些经历称为经验（*experience*）。

经验回放的基本思想是将 *Agent* 的经历存储在一个经验池（*experience pool*）中，然后在训练过程中从经验池中随机采样经验进行训练。这样做的好处是可以减少数据之间的相关性，提升训练效率。

经验回放的一个缺点是需要额外的存储空间来存储经验池，在经验池较大时可能会导致内存不足的问题。


## Q-learning

Q-table：Q-table 是一种二维表格，用于存储 Agent 在不同状态和动作下的期望奖励。Q-table 算法的基本思想是使用贪心策略来选择当前状态下最优的动作，并在执行动作后更新 Q-table 中的值。

### 行动值 (action value)

行动值是 Agent 在给定状态和动作下执行该动作的预期奖励。在 Q-learning 算法中，行动值是存储在 Q-table 中的，并用于更新 Q-table 的值。

### 探索率 (exploration rate)

> 探索率就是指系统会采取随机行为的概率.

在强化学习（*reinforcement learning*）中，探索率是 Agent 采取随机动作的概率。在 Q-learning 算法中，Agent 在较早的训练阶段需要更多地探索，以了解不同的动作和状态的奖励。随着训练的进行，Agent 可以逐渐降低探索率，以便更多地采用已经学到的行为。探索率（*exploration rate*）通常指的是一个参数，用于控制 Agent 在尝试新的行为或者采用已知的最优行为的概率。在许多强化学习算法中，探索率常常使用一个符号为 `ε` 的参数来表示，这个参数被称为 ε-贪心策略（ε-greedy strategy）。

**ε-贪心策略**（*ε-greedy strategy*）是一种用于强化学习（reinforcement learning）的策略，用于控制 Agent 在尝试新的行为或者采用已知的最优行为的概率。在 ε-贪心策略中，**Agent 会以 `1-ε` 的概率采用最优的行为，以 `ε` 的概率选择随机的行为**。这样做的原因是，在探索新的策略的同时，Agent 也可以在大多数情况下采用最优的行为。

### 折扣率 (discount rate)

折扣率是一个小于 1 的数字，用于表示未来的奖励的价值比现在的小。在 Q-learning 算法中，折扣率用于计算未来的期望奖励，并用于更新 Q-table 的值。

### 学习率 (learning rate)

学习率是一个小于 1 的数字，用于表示当前的经验对于更新 Q-table 的值的重要程度。在 Q-learning 算法中，学习率决定了新的经验如何影响 Q-table 的值。


### Other

经验池（experience pool）和 Q-table 是两种不同的数据结构，但都可以用于存储 Agent 在强化学习中的经验。

Q-table 是一种二维表格，用于存储 Agent 在不同状态和动作下的期望奖励。Q-table 算法的基本思想是使用贪心策略来选择当前状态下最优的动作，并在执行动作后更新 Q-table 中的值。

经验池（experience pool）是一种数据结构，用于存储 Agent 经历的序列，每个经历包含三个元素：当前状态、执行的动作和获得的奖励。经验池可以用于经验回放（experience replay）算法中，从中随机采样经验进行训练。

Q-table 算法和经验回放算法都是强化学习的基本算法，但它们有各自的优缺点。Q-table 算法适用于状态空间较小的情况，因为 Q-table 的大小与状态空间的大小成正比。经验回放算法适用于状态空间较大的情况，因为它可以减少数据之间的相关性，提升训练效率。




In [25]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        """
        初始化模型参数
        state_size:代理所有可能所处的状态(state)总数
        action_size:代理所有可以执行的动作(action)总数
        """
        self.state_size = state_size
        self.action_size = action_size

        # 使用deque容器存储每次的<状态,动作,奖励>等信息
        self.memory = deque(maxlen=2000)
        
        # discount rate 折扣率
        self.gamma = 0.95

        # exploration rate 探索率
        self.epsilon = 1.0
        # 最小学习率
        self.epsilon_min = 0.01
        # 学习率衰减
        self.epsilon_decay = 0.99

        # learning rate 学习率
        self.learning_rate = 0.001
        
        # 这将会创建两个model
        # self.model:
        # self.target_model:
        self.model = self._build_model()
        self.target_model = self._build_model()

        self.update_target_model()


    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        """
        实现自定义Huber损失函数 loss function

        err = y_true - y_pred
        ret = 0
        if abs(error) <= clip_delta :
            ret = 0.5 * err**2
        else:
            ret = clip_delta * (abs(err) - 0.5 * clip_delta)
        return 
        """
        
        error = y_true - y_pred
        cond  = tf.keras.backend.abs(error) <= clip_delta

        squared_loss = 0.5 * tf.keras.backend.square(error)
        quadratic_loss = 0.5 * tf.keras.backend.square(clip_delta) + \
                        clip_delta * (tf.keras.backend.abs(error) - clip_delta)

        return tf.keras.backend.mean(tf.where(cond, squared_loss, quadratic_loss))


    def _build_model(self):
        """
        创建神经网络模型
        """

        """
        input_layer:  输入尺寸state_size,即代理(agent)当前所处的状态(state)
        hidden_layer: 24个单元,ReLU激活
        output_layer: 输出尺寸为action_size,即代理(agent)可以执行的动作(action),linear激活
        """
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(units=24, input_dim=self.state_size, activation='relu'))
        model.add(tf.keras.layers.Dense(units=24, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_size, activation='linear'))
        
        """
        loss:使用自定义的loss函数'_huber_loss'
        optimizer:使用随机梯度下降(Adam)作为优化器
        """
        model.compile(loss=self._huber_loss,
                      optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model


    def update_target_model(self):
        """
        更新模型权重
        copy weights from model to target_model
        """
        self.target_model.set_weights(self.model.get_weights())


    def memorize(self, state, action, reward, next_state, done):
        """
        存储信息 (状态、动作、奖励、下一次状态、游戏是否结束)
        """
        self.memory.append((state, action, reward, next_state, done))


    def act(self, state):
        """
        ......
        """
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action


    def replay(self, batch_size):
        """

        """

        """
        sample() 是从序列或集合中选择 k 个唯一的随机元素。
        此处即从'memory'中随机抽取batch_size个(默认为32)条数据信息
        """
        minibatch = random.sample(self.memory, batch_size)

        # with tf.device('/gpu:0'):

        for state, action, reward, next_state, done in minibatch:
            
            target = self.model.predict(state)

            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]

            # state = tf.constant(state, dtype=tf.float32, shape=[1, self.state_size])
            # target = tf.constant(target, dtype=tf.float32, shape=[1, self.action_size])

            self.model.fit(state, target, epochs=1, verbose=0)
        

        """
        当 探索率(epsilon) 大于 设定的 最小探索率(epsilon_min) 时,
        将当前的 探索率 乘上一个 衰减项(epsilon_decay).
        """
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def load(self, name):
        """
        加载已有模型
        """
        self.model.load_weights(name)


    def save(self, name):
        """
        保存模型
        """
        self.model.save_weights(name)


在CartPole-v1模拟环境中,

***state***是一个四维的向量.这些信息组合起来,描述了小车杆系统的当前状态.

```
x: 小车的水平位移,从小车的初始位置(位于屏幕中央)开始测量.
x_dot: 小车的水平速度。
theta: 杆的倾角,从垂直向下的位置开始测量.
theta_dot: 杆的角速度.
```





这个任务的目标是通过控制小车的水平力量，使得杆保持垂直,并尽量延长运行时间.

In [26]:
EPISODES = 5

# create environment
env = gym.make('CartPole-v1')

# 获取环境中的可能的状态数量
state_size = env.observation_space.shape[0]

# 获取环境中的可采取的动作数量
action_size = env.action_space.n

# 创建代理
agent = DQNAgent(state_size, action_size)
#  加载已有权重模型
# agent.load("./save/cartpole-ddqn.h5")

done = False

batch_size = 32

for i in range(EPISODES):
    
    # 重置环境,获取当前的状态
    state, _ = env.reset()
    
    # print("\n state:", state, np.shape(state))

    state = np.reshape(state, [1, state_size])

    print("\n state:", state, np.shape(state))

    # 每一个新的环境中执行500次动作
    for time in range(10):

        # env.render()
        """
        render() 函数用于渲染出当前的智能体以及环境的状态
        """

        action = agent.act(state)
        """ 根据当前环境做出动作 """
        
        next_state, reward, done, _, _ = env.step(action)
        """
        在环境中执行当前动作得到反馈信息 
        next_state: 下一个状态信息
        reward:     执行动作后得到的奖励
        done:       一个布尔值,<表示本次游戏是否已经结束>.
                    :当木棒倾斜超过特定角度或者小车的位置超
                    出轨道边界时,游戏结束.
        """
        
        # reward = reward if not done else -10

        x, x_dot, theta, theta_dot = next_state
        """
        解析`next_state`的信息, 并重新计算reward
        x:小车的水平位移,从小车的初始位置(位于屏幕中央)开始测量.
        x_dot:小车的水平速度。
        theta:杆的倾角,从垂直向下的位置开始测量.
        theta_dot:杆的角速度.
        """
        

        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        reward = r1 + r2
        
        next_state = np.reshape(next_state, [1, state_size])
        

        agent.memorize(state, action, reward, next_state, done)
        
        # 状态转换
        state = next_state
        
        print('.', end='')
        
        # 游戏结束：
        if done:
            # 更新权重模型
            agent.update_target_model()

            print("episode: {}/{}, score: {}, e: {:.2}"
                    .format(i, EPISODES, time, agent.epsilon))
            """
            打印信息,格式: episode: 1/5, score: 36, e: 0.87 
            episode:
            score:游戏得分
            epsilon:探索率
            """
            
            break
        
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        """
        ......
        """

    # if e % 10 == 0:
    #     agent.save("./save/cartpole-ddqn.h5")


 state: [[ 0.03058243 -0.03656648 -0.02251788 -0.0416424 ]] (1, 4)
..........
 state: [[-0.04189845 -0.04963819 -0.02266778  0.04613419]] (1, 4)
..........
 state: [[-0.01011473  0.03153153  0.02147594  0.01707859]] (1, 4)
..........
 state: [[0.01431751 0.04364578 0.00816001 0.0305667 ]] (1, 4)
1/1 [==============================] - 0s 24ms/step

 state: [[ 0.02707659  0.02302776 -0.01193946 -0.00625541]] (1, 4)
1/1 [==============================] - 0s 25ms/step

 ok


In [22]:

print(tf.config.list_physical_devices('GPU'))

# # config = tf.ConfigProto()

    # print("asdasd")
    # pass
#     model = tf.keras.Sequential();
#     for state, 

# import tensorflow as tf

# # 设置 GPU 选项
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

# 将模型和数据加载到 GPU 上
# with tf.device('/gpu:0'):
#   model = Model()  # 定义模型
#   for state, target in zip(states, targets):
#     state = tf.constant(state, dtype=tf.float32, shape=(1, state_size))
#     target = tf.constant(target, dtype=tf.float32, shape=(1, target_size))
#     model.fit(state, target, epochs=1, verbose=0)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
asdasd
